In [ ]:
!pip install -q pandas nltk scikit-learn vaderSentiment "transformers[sentencepiece]" torch

import pandas as pd
import nltk
import re # for regular expressions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import torch

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
if not torch.cuda.is_available():
    print("WARNING: GPU is NOT enabled. Classification will be very slow.")

--- Environment and Libraries Ready ---


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df = pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv",index_col=0)

In [ ]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23486 entries, 0 to 23485
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Clothing ID              23486 non-null  int64 
 1   Age                      23486 non-null  int64 
 2   Title                    19676 non-null  object
 3   Review Text              22641 non-null  object
 4   Rating                   23486 non-null  int64 
 5   Recommended IND          23486 non-null  int64 
 6   Positive Feedback Count  23486 non-null  int64 
 7   Division Name            23472 non-null  object
 8   Department Name          23472 non-null  object
 9   Class Name               23472 non-null  object
dtypes: int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df.isnull().sum()

,0
Clothing ID,0
Age,0
Title,3810
Review Text,845
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,14
Department Name,14
Class Name,14


In [ ]:
df.dropna(subset=["Review Text"],inplace = True)

In [ ]:
df.isnull().sum()

,0
Clothing ID,0
Age,0
Title,2966
Review Text,0
Rating,0
Recommended IND,0
Positive Feedback Count,0
Division Name,13
Department Name,13
Class Name,13


In [ ]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
 df["Title"].fillna('') + df["Review Text"]

,0
0,Absolutely wonderful - silky and sexy and comf...
1,Love this dress! it's sooo pretty. i happene...
2,Some major design flawsI had such high hopes f...
3,"My favorite buy!I love, love, love this jumpsu..."
4,Flattering shirtThis shirt is very flattering ...
...,...
23481,Great dress for many occasionsI was very happy...
23482,Wish it was made of cottonIt reminds me of mat...
23483,"Cute, but see throughThis fit well, but the to..."
23484,"Very cute dress, perfect for summer parties an..."


In [ ]:
df["full_review"]  =  df["Title"].fillna('') + df["Review Text"]

In [ ]:
df[["full_review","Rating"]].head()

,full_review,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,Some major design flawsI had such high hopes f...,3
3,"My favorite buy!I love, love, love this jumpsu...",5
4,Flattering shirtThis shirt is very flattering ...,5


In [ ]:
# Initialize the lemmatizer and stop words list

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):

    text = text.lower()

    # Remove punctuation (and numbers, if desired)
    text = re.sub(r'[^a-z\s]', '', text)


    tokens = word_tokenize(text)


    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]


    return " ".join(cleaned_tokens)




In [ ]:
df['cleaned_review'] = df['full_review'].apply(preprocess_text)

In [ ]:
df['full_review'].head()

,full_review
0,Absolutely wonderful - silky and sexy and comf...
1,Love this dress! it's sooo pretty. i happene...
2,Some major design flawsI had such high hopes f...
3,"My favorite buy!I love, love, love this jumpsu..."
4,Flattering shirtThis shirt is very flattering ...


In [ ]:
df['cleaned_review'].head()

,cleaned_review
0,absolutely wonderful silky sexy comfortable
1,love dress sooo pretty happened find store im ...
2,major design flawsi high hope dress really wan...
3,favorite buyi love love love jumpsuit fun flir...
4,flattering shirtthis shirt flattering due adju...


In [ ]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,full_review,cleaned_review
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,Absolutely wonderful - silky and sexy and comf...,absolutely wonderful silky sexy comfortable
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,Love this dress! it's sooo pretty. i happene...,love dress sooo pretty happened find store im ...
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,Some major design flawsI had such high hopes f...,major design flawsi high hope dress really wan...
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"My favorite buy!I love, love, love this jumpsu...",favorite buyi love love love jumpsuit fun flir...
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,Flattering shirtThis shirt is very flattering ...,flattering shirtthis shirt flattering due adju...


In [ ]:
# Calculate VADER score first
analyzer = SentimentIntensityAnalyzer()
tqdm.pandas(desc="Calculating VADER Scores")
df['vader_score'] = df['cleaned_review'].progress_apply(lambda r: analyzer.polarity_scores(r)['compound'])

Calculating VADER Scores:   0%|          | 0/22641 [00:00<?, ?it/s]

In [ ]:
df[["cleaned_review","vader_score","Rating","Recommended IND"]].head()

,cleaned_review,vader_score,Rating,Recommended IND
0,absolutely wonderful silky sexy comfortable,0.8991,4,1
1,love dress sooo pretty happened find store im ...,0.9710,5,1
2,major design flawsi high hope dress really wan...,0.9081,3,0
3,favorite buyi love love love jumpsuit fun flir...,0.9587,5,1
4,flattering shirtthis shirt flattering due adju...,0.9274,5,1


In [ ]:
def get_ultimate_sentiment(row):
    """
    Assigns sentiment based on a hierarchy of signals:
    1. Explicit negative signals (Rating 1-2 or Not Recommended) are always 'Negative'.
    2. Explicit positive signals (Rating 4-5) are 'Positive'.
    3. Ambiguous 3-star reviews are decided by the VADER score of the text.
    """
    rating = row['Rating']
    recommended = row['Recommended IND']
    vader_score = row['vader_score']

    # Rule 1
    if rating <= 2 or recommended == 0:
        return 'Negative'

    # Rule 2
    if rating >= 4:
        return 'Positive'

    # Rule 3

    if rating == 3:
        if vader_score <= -0.05:
            return 'Negative'  # review with negative language
        elif vader_score >= 0.05:
            return 'Positive' #  review with positive language
        else:
            return 'Neutral'   # neutral 3-star review

    # Fallback case
    return 'Neutral'

df['sentiment_category'] = df.apply(get_ultimate_sentiment, axis=1)
print("\nDefinitive sentiment analysis complete.")
print("New ULTIMATE Sentiment Distribution:")
print(df['sentiment_category'].value_counts())


Definitive sentiment analysis complete.
New ULTIMATE Sentiment Distribution:
sentiment_category
Positive    18365
Negative     4257
Neutral        19
Name: count, dtype: int64


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:

print("\nStarting topic modeling...")
vectorizer = CountVectorizer(max_df=0.9, min_df=5, stop_words='english')
doc_term_matrix = vectorizer.fit_transform(df['cleaned_review'])

num_topics = 5
lda = LatentDirichletAllocation(n_components=num_topics, learning_method='online', n_jobs=-1, random_state=42)
lda.fit(doc_term_matrix)
print("Topic modeling complete.")


Starting topic modeling...
Topic modeling complete.


In [ ]:
# Get the vocabulary
feature_names = vectorizer.get_feature_names_out()

# display the top words for each topic
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx + 1}:")
        # Get the indices of the top words
        top_words_indices = topic.argsort()[:-num_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_indices]
        print(" ".join(top_words))

print("\n--- Discovered Topics ---")
display_topics(lda, feature_names, 10)


--- Discovered Topics ---
Topic 1:
sweater look like fabric shape sleeve warm quality material looking
Topic 2:
size fit small like im run ordered large look waist
Topic 3:
color beautiful pretty love blouse shirt white fabric lace green
Topic 4:
dress size fit love store wear beautiful im retailer online
Topic 5:
great love wear fit perfect comfortable jean soft color look


In [ ]:
topic_mapping = {
    0: "Dresses & General Style",
    1: "Blouses & Tops Fit",
    2: "Sweaters & Material Feel",
    3: "General Positive Feedback",
    4: "Sizing & Fit Issues"
}
print(f"\nTopic mapping created: {topic_mapping}")



Topic mapping created: {0: 'Dresses & General Style', 1: 'Blouses & Tops Fit', 2: 'Sweaters & Material Feel', 3: 'General Positive Feedback', 4: 'Sizing & Fit Issues'}


In [ ]:

from transformers import pipeline
from tqdm.auto import tqdm
import time

if torch.cuda.is_available():
    device = 0
    torch_dtype = torch.float16  # Use half-precision for speed
    print(f"\n Using GPU: {torch.cuda.get_device_name(0)} with fp16")
else:
    device = -1
    torch_dtype = None
    print("\n No GPU found, running on CPU (will be slow)")

hf_classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=device,
    torch_dtype=torch_dtype
)



✅ Using GPU: Tesla T4 with fp16


Device set to use cuda:0


In [ ]:
# Select the DataFrame of reviews that our ULTIMATE sentiment model classified as Negative
negative_reviews_df = df[df['sentiment_category'] == 'Negative'].copy()
candidate_labels = ["Sizing/Fit", "Fabric/Quality", "Color/Appearance", "Construction/Durability", "Price/Value", "Other"]

print(f"Classifying {len(negative_reviews_df)} truly negative reviews...")

# Convert the review texts to a list for efficient processing
review_list = negative_reviews_df['Review Text'].tolist()


classified_results = []
batch_size = 64

for i in tqdm(range(0, len(review_list), batch_size), desc="🚀 Classifying Batches"):
    batch = review_list[i:i+batch_size]
    # multi_label=False only one category is chosen
    results = hf_classifier(batch, candidate_labels, multi_label=False)
    classified_results.extend(results)

print("\n Classification complete.")

Classifying 4257 truly negative reviews...


🚀 Classifying Batches:   0%|          | 0/67 [00:00<?, ?it/s]


✅ Classification complete.


In [ ]:
temp_business_categories = [result['labels'][0] for result in classified_results]
print("📊 Negative Feedback Category Distribution:")
print(pd.Series(temp_business_categories).value_counts())

📊 Negative Feedback Category Distribution:
Sizing/Fit                 1518
Fabric/Quality             1113
Color/Appearance           1026
Other                       457
Price/Value                 137
Construction/Durability       6
Name: count, dtype: int64


In [ ]:


# ---  Assign Topic Names ---
topic_assignments = lda.transform(doc_term_matrix).argmax(axis=1)
df['topic_number'] = topic_assignments
df['topic_name'] = df['topic_number'].map(topic_mapping)
print(f"Reviews with unmapped topics: {df['topic_name'].isnull().sum()}")


# ---  Safely Merge the AI Business Categories ---
business_category_series = pd.Series(
    data=[result['labels'][0] for result in classified_results],
    index=negative_reviews_df.index,
    name='business_category' # Explicitly name the Series
)


df = df.merge(business_category_series.to_frame(), left_index=True, right_index=True, how='left')
print("AI-generated business categories have been merged.")


print("\nColumns in DataFrame after merge:")
print(df.columns)


final_df = df[[
    'Clothing ID',
    'Age',
    'Department Name',
    'Rating',
    'Recommended IND',
    'full_review',
    'sentiment_category',
    'topic_name',
    'business_category'
]].copy()

final_df.rename(columns={
    'Recommended IND': 'Recommended',
    'full_review': 'Review Text',
    'sentiment_category': 'Sentiment',
    'topic_name': 'Review Topic',
    'business_category': 'Negative Feedback Category'
}, inplace=True)
print("\nFinal columns selected and renamed for clarity.")


output_filename = 'Product_Review_Analysis_Final.csv'
final_df.to_csv(output_filename, index=False)

--- Step 5: Consolidating all insights for the final dashboard file ---
Reviews with unmapped topics: 0
AI-generated business categories have been merged.

Columns in DataFrame after merge:
Index(['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name', 'full_review', 'cleaned_review',
       'vader_score', 'sentiment_category', 'topic_number', 'topic_name',
       'business_category_x', 'business_category_y', 'business_category'],
      dtype='object')

Final columns selected and renamed for clarity.


In [ ]:
print(f"\n data is ready in '{output_filename}'")
print("\nDisplaying a sample of the final data:")
final_df.head(20)


🎉 SUCCESS! Your definitive data is ready in 'Product_Review_Analysis_Final.csv'

Displaying a sample of the final data:


,Clothing ID,Age,Department Name,Rating,Recommended,Review Text,Sentiment,Review Topic,Negative Feedback Category
0,767,33,Intimate,4,1,Absolutely wonderful - silky and sexy and comf...,Positive,Sizing & Fit Issues,NaN
1,1080,34,Dresses,5,1,Love this dress! it's sooo pretty. i happene...,Positive,General Positive Feedback,NaN
2,1077,60,Dresses,3,0,Some major design flawsI had such high hopes f...,Negative,Blouses & Tops Fit,Sizing/Fit
3,1049,50,Bottoms,5,1,"My favorite buy!I love, love, love this jumpsu...",Positive,Sizing & Fit Issues,NaN
4,847,47,Tops,5,1,Flattering shirtThis shirt is very flattering ...,Positive,Sizing & Fit Issues,NaN
5,1080,49,Dresses,2,0,Not for the very petiteI love tracy reese dres...,Negative,Sweaters & Material Feel,Color/Appearance
6,858,39,Tops,5,1,Cagrcoal shimmer funI aded this in my basket a...,Positive,Sweaters & Material Feel,NaN
7,858,39,Tops,4,1,"Shimmer, surprisingly goes with lotsI ordered ...",Positive,Blouses & Tops Fit,NaN
8,1077,24,Dresses,5,1,FlatteringI love this dress. i usually get an ...,Positive,Blouses & Tops Fit,NaN
9,1077,34,Dresses,5,1,"Such a fun dress!I'm 5""5' and 125 lbs. i order...",Positive,Blouses & Tops Fit,NaN
